**AutoML OSS入門（4）**

# AutoMLを最短3行で！表形式や画像、テキストのデータにも対応可能なOSS「AutoGluon」

本ノートブックの紹介記事と併せてご覧ください。
- [＠IT連載 AutoML OSS入門（4）- 第4回「AutoMLを最短3行で！表形式や画像、テキストのデータにも対応可能なOSS「AutoGluon」」](https://atmarkit.itmedia.co.jp/ait/articles/2109/27/news011.html)

また、ノートブックの扱い方やタイタニックの生存予測データについては、連載記事の第1回も参照してください。
- [＠IT連載 AutoML OSS入門（1）- 第1回「機械学習モデル構築作業の煩雑さを解消する「AutoML」とは――歴史、動向、利用のメリットを整理する」](https://www.atmarkit.co.jp/ait/articles/2107/02/news006.html)

## MNISTデータでAutoML
MNISTという画像分類用データを使って、AutoGluonを紹介していきます。

### セットアップ  
まず、ノートブックの「ランタイム」タブから「ランタイムのタイプの変更」をクリックし、GPUを選択します。
その後、Colab上でAutoGluonを動かすために、セットアップを行います。インストールが終わったらColabのランタイムを再起動しましょう。

In [ ]:
!pip install -U pip
!pip install -U setuptools wheel
!pip install -U "mxnet_cu101<2.0.0"
!pip install autogluon
!pip install timm
!pip install torch torchvision

     |████████████████████████████████| 1.6 MB 12.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 816 kB 10.9 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 356.7 MB 32 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 352 kB 11.3 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 273 kB 48.8 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 22.3 MB 1.7 MB/s 
     |████████████████████████████████| 206 kB 45.1 MB/s 
     |████████████████████████████████| 4.2 MB 46.3 MB/s 
     |████████████████████████████████| 27.4 MB 100 kB/s 
     |████████████████████████████████| 131 kB 50.4 MB/s 
     |████████████████████████████████| 779 kB 51.4 MB/s 
     |████████████████████████████████| 1.3 MB 45.4 MB/s 
     |████████████████████████████████| 119 kB 50.8 MB/s 
     |████████████████████████████████| 3.0 MB 46.5 MB/s 
     |███████████████████████

     |████████████████████████████████| 376 kB 12.7 MB/s 


(注) ランタイムを再起動してください。
- ランタイムの再起動方法<br>
ランタイム > ランタイムを再起動 を実行


### ライブラリーのインポート

画像分類を行うための`ImagePredictor`クラスをインポートします。

In [ ]:
from autogluon.vision import ImagePredictor

/usr/local/lib/python3.7/dist-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.8.0` and `torch==1.9.0+cu102` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


In [ ]:
# バージョンの確認
from autogluon.vision.version import __version__
print(__version__)

0.3.1


### データのロード

MNISTデータをロードします。

In [ ]:
import os
import torchvision

In [ ]:
root_dir = '/MNIST'
train_data = torchvision.datasets.MNIST(root=root_dir, train=True, download=True)
test_data = torchvision.datasets.MNIST(root=root_dir, train=False, download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /MNIST/MNIST/raw/train-images-idx3-ubyte.gz to /MNIST/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to /MNIST/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to /MNIST/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to /MNIST/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


今回はフォルダからデータを読み込むパターンを試したいので、一度画像として保存します。

In [ ]:
# ディレクトリ作成
for i in range(10):
    os.makedirs('/MNIST/train/{}'.format(i), exist_ok=True)
    os.makedirs('/MNIST/test/{}'.format(i), exist_ok=True)

# 画像保存
num = 0
for img, label in train_data:
    img.save('/MNIST/train/{0}/{1}.png'.format(str(label), str(num).zfill(5)))
    num = num + 1

num = 0
for img, label in test_data:
    img.save('/MNIST/test/{0}/{1}.png'.format(str(label), str(num).zfill(5)))
    num = num + 1

MNISTディレクトリに以下2つのフォルダを作成しました。

*   train: 学習用のデータフォルダ
*   test: 予測用のデータフォルダ


MNISTデータをフォルダから読み込みます。

In [ ]:
train_df, _, test_df = ImagePredictor.Dataset.from_folders(
    '/MNIST/', train='train', test='test')
print(train_df)

                          image  label
0      /MNIST/train/0/00001.png      0
1      /MNIST/train/0/00021.png      0
2      /MNIST/train/0/00034.png      0
3      /MNIST/train/0/00037.png      0
4      /MNIST/train/0/00051.png      0
...                         ...    ...
59995  /MNIST/train/9/59948.png      9
59996  /MNIST/train/9/59969.png      9
59997  /MNIST/train/9/59973.png      9
59998  /MNIST/train/9/59990.png      9
59999  /MNIST/train/9/59992.png      9

[60000 rows x 2 columns]


### モデルの学習
AutoGluonなら1行でモデル構築が可能です。データの前処理については内部で実施してくれます。  
※今回は30分の時間制限を設定しています。

In [ ]:
predictor = ImagePredictor().fit(train_df, time_limit=1800)

Reset labels to [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
NumExpr defaulting to 2 threads.
Randomly split train_data into train[54000]/validation[6000] splits.
The number of requested GPUs is greater than the number of available GPUs.Reduce the number to 1
Starting fit without HPO
modified configs(<old> != <new>): {
root.train.early_stop_max_value 1.0 != inf
root.train.batch_size 32 != 16
root.train.epochs    200 != 15
root.train.early_stop_baseline 0.0 != -inf
root.train.early_stop_patience -1 != 10
root.img_cls.model   resnet101 != resnet50
root.misc.num_workers 4 != 2
root.misc.seed       42 != 235
}
Saved config to /content/87df2919/.trial_0/config.yaml
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet50_ram-a26f946b.pth" to /root/.cache/torch/hub/checkpoints/resnet50_ram-a26f946b.pth
Model resnet50 created, param count:                                         23528522
AMP not enabled. Training in float32.
Disable EMA as it is not supported 

In [ ]:
predictor.fit_summary()

{'best_config': {'batch_size': 16,
  'dist_ip_addrs': None,
  'early_stop_baseline': -inf,
  'early_stop_max_value': inf,
  'early_stop_patience': 10,
  'epochs': 15,
  'final_fit': False,
  'gpus': [0],
  'log_dir': '/content/87df2919',
  'lr': 0.01,
  'model': 'resnet50',
  'ngpus_per_trial': 8,
  'nthreads_per_trial': 128,
  'num_trials': 1,
  'num_workers': 2,
  'problem_type': 'multiclass',
  'scheduler': 'local',
  'search_strategy': 'random',
  'searcher': 'random',
  'seed': 235,
  'time_limits': 1800,
  'wall_clock_tick': 1632118353.518508},
 'fit_history': {'best_config': {'batch_size': 16,
   'dist_ip_addrs': None,
   'early_stop_baseline': -inf,
   'early_stop_max_value': inf,
   'early_stop_patience': 10,
   'epochs': 15,
   'final_fit': False,
   'gpus': [0],
   'log_dir': '/content/87df2919',
   'lr': 0.01,
   'model': 'resnet50',
   'ngpus_per_trial': 8,
   'nthreads_per_trial': 128,
   'num_trials': 1,
   'num_workers': 2,
   'problem_type': 'multiclass',
   'scheduler

テスト用データでモデルの評価を行います。

In [ ]:
test_acc = predictor.evaluate(test_df)
print(test_acc)

[Epoch 0] validation: top1=0.963500 top5=0.999400


{'loss': 0.22871450762152673, 'top1': 0.9635, 'top5': 0.9994}


以上でAutoGluonを使用したMNISTの画像分類は終了です。